In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score

from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from scipy.optimize import minimize

/Users/khangtran/anaconda3/envs/torch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Parameters

In [2]:
seed = 1
folds = 5
num_feat = 16
epoch = 100
# lr = 0.5
alpha = 0.5
mode = 'fair'
num_draw = 100000

In [3]:
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## 2. Utils

In [4]:
def fold_separation(train_df, folds, feat_cols, label):
    skf = StratifiedKFold(n_splits=folds)
    train_df['fold'] = np.zeros(train_df.shape[0])
    for i, (idxT, idxV) in enumerate(skf.split(train_df[feat_cols], train_df[label])):
        train_df.at[idxV, 'fold'] = i

## 3. Read Data

In [5]:
df = pd.read_csv('Data/Bank/formated_bank.csv')
# df['bias'] = np.ones(df.shape[0])
feature_cols = list(df.columns)
feature_cols.remove('y')
feature_cols.remove('z')
feature_cols.remove('label')
feature_cols.remove('is_train')
feature_cols.remove('intercept')
label = 'y'
train_df = df[df['is_train'] == 1].reset_index(drop=True).sample(frac = 1)
test_df = df[df['is_train'] == 0].reset_index(drop=True).sample(frac = 1)
male_df = train_df[train_df['z'] == 1].copy().reset_index(drop=True)
female_df = train_df[train_df['z'] == 0].copy().reset_index(drop=True)
fold_separation(male_df, folds, feature_cols, label)
fold_separation(female_df, folds, feature_cols, label)
train_df = pd.concat([male_df, female_df], axis=0).reset_index(drop=True)

In [6]:
male_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,campaign,pdays,previous,poutcome,z,label,intercept,y,is_train,fold
0,73,2.078277,0.171004,-1.442665,0.13576,0.307118,1.052817,0.434409,0.408136,1.674430,...,-0.236331,-0.476830,-0.371080,-0.470990,1,1,0.0,1,1,0.0
1,84,2.078277,-0.795053,-0.420689,0.13576,26.329134,1.052817,0.434409,0.408136,1.435485,...,-0.545439,2.712669,0.562012,0.489414,1,1,0.0,1,1,0.0
2,61,2.078277,-0.795053,-0.420689,0.13576,-0.424768,1.052817,0.434409,0.681550,1.435485,...,-0.545439,1.371861,0.562012,0.091106,1,0,0.0,0,1,0.0
3,82,2.078277,-0.795053,-0.420689,0.13576,-0.363090,1.052817,0.434409,0.408136,0.121292,...,-0.545439,-0.476830,-0.371080,-0.470990,1,1,0.0,1,1,0.0
4,71,-0.097838,-0.795053,1.371580,0.13576,-0.327798,1.052817,0.434409,0.681550,1.316013,...,0.072777,-0.476830,-0.371080,-0.470990,1,1,0.0,1,1,0.0


## 4. Normal Logistic Regression (Sklearn)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
clf = LogisticRegression(random_state=0, max_iter=epoch, fit_intercept=False, verbose=1).fit(X=male_df[feature_cols].values, y=male_df[label].values)
clf.score(X=male_df[feature_cols].values, y=male_df[label].values)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           16     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.89229D+02    |proj g|=  4.93350D+03

At iterate   50    f=  1.33082D+02    |proj g|=  3.00376D+01

At iterate  100    f=  1.32394D+02    |proj g|=  3.57465D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   16    100    121      1     0     0   3.575D+00   1.324D+02
  F =   132.39432796144797     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.
/Users/khangtran/anaconda3/envs/torch/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


0.7875457875457875

In [8]:
y_pred = clf.predict_proba(X=male_df[feature_cols].values)[:,1]
log_loss(y_true=male_df[label], y_pred=y_pred)

0.47017630412741335

In [9]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=epoch, fit_intercept=False).fit(X=female_df[feature_cols].values, y=female_df[label].values)
clf.score(X=female_df[feature_cols].values, y=female_df[label].values)

0.9362363919129082

In [10]:
y_pred = clf.predict_proba(X=female_df[feature_cols].values)[:,1]
log_loss(y_true=female_df[label], y_pred=y_pred)

0.23368934710215286

In [11]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=epoch, fit_intercept=False).fit(X=train_df[feature_cols].values, y=train_df[label].values)
clf.score(X=test_df[feature_cols].values, y=test_df[label].values)

0.8986206896551724

## 5. Logistic Regression with Taylor approximation

In [12]:
feature_cols

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome']

### 5.1 With pytorch

In [13]:
# female
X_fem = female_df[feature_cols].values
y_fem = female_df[label].values.reshape(-1, 1)
X_fem = X_fem/np.linalg.norm(X_fem, ord=2, axis=1).reshape(-1, 1)

# male
X_mal = male_df[feature_cols].values
y_mal = male_df[label].values.reshape(-1, 1)
X_mal = X_mal/np.linalg.norm(X_mal, ord=2, axis=1).reshape(-1, 1)

# train
X = train_df[feature_cols].values
y = train_df[label].values.reshape(-1, 1)
X = X/np.linalg.norm(X, ord=2, axis=1).reshape(-1, 1)

In [14]:
def get_coefficient(X, y, epsilon=None, lbda=None, mode='torch'):
    num_data_point = X.shape[0]
    num_feat = X.shape[1]
    sensitivity = num_feat**2/4 + 3*num_feat
#     lbda = sensitivity*4/epsilon
    coff_0 = 1.0
    coff_1 = np.sum(X/2 - X*y, axis = 0).astype(np.float32)
    coff_2 = np.dot(X.T, X).astype(np.float32)
    noise_1 = np.random.laplace(0.0, sensitivity/epsilon, coff_1.shape).astype(np.float32)
    noise_2 = np.random.laplace(0.0, sensitivity/epsilon, coff_2.shape).astype(np.float32)
    if mode == 'scipy':
        return coff_0, (1/num_data_point)*coff_1.reshape(-1, 1), (1/num_data_point)*coff_2
    elif mode == 'scipy_dp':
        coff_1 = coff_1.reshape(-1, 1) 
        coff_1 = coff_1 + noise_1
        coff_2 = coff_2 + np.triu(noise_2, k=0) + np.triu(noise_2, k=1).T + lbda*np.identity(num_feat)
        return coff_0, (1/num_data_point)*coff_1, (1/num_data_point)*coff_2
    elif mode == 'torch' or mode == 'fair':
        return coff_0, (1/num_data_point)*torch.from_numpy(coff_1.reshape(-1, 1)), (1/num_data_point)*torch.from_numpy(coff_2)
    elif mode == 'func' or mode == 'fairdp':
        coff_1 = coff_1 + noise_1
        coff_2 = coff_2 + np.triu(noise_2, k=0) + np.triu(noise_2, k=1).T #+ lbda*np.identity(num_feat)
        w, V = np.linalg.eig(coff_2)
        indx = np.where(w > 0)[0]
        w = w[indx].astype(np.float32)
        V = V[indx, :].astype(np.float32)
        coff_2 = np.identity(len(w))*w.astype(np.float32)
        coff_1 = np.dot(coff_1, V.T).astype(np.float32)
        return coff_0, (1/num_data_point)*torch.from_numpy(coff_1.reshape(-1, 1)), (1/num_data_point)*torch.from_numpy(coff_2), V, 

In [15]:
eps = 10.0
lbd = 0.0
lr = 0.05

In [16]:
if (mode == 'func') or (mode =='fairdp'):
    f_coff_0, f_coff_1, f_coff_2, Q_f = get_coefficient(X=X_fem, y=y_fem, epsilon=eps, lbda=lbd, mode=mode)
    m_coff_0, m_coff_1, m_coff_2, Q_m = get_coefficient(X=X_mal, y=y_mal, epsilon=eps, lbda=lbd, mode=mode)
else:
    f_coff_0, f_coff_1, f_coff_2 = get_coefficient(X=X_fem, y=y_fem, epsilon=eps, lbda=lbd, mode=mode)
    m_coff_0, m_coff_1, m_coff_2 = get_coefficient(X=X_mal, y=y_mal, epsilon=eps, lbda=lbd, mode=mode)

In [17]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [18]:
if mode == 'func' or mode == 'fairdp':
    print(Q_m.shape, Q_f.shape)
print(check_symmetric(m_coff_2),check_symmetric(f_coff_2))

True True


In [19]:
f_coff_1.size()

torch.Size([16, 1])

In [20]:
if mode == 'fair' or mode == 'fairdp':
    model_mal = torch.randn((num_feat, num_draw), requires_grad=True).float()
    model_fem = torch.randn((num_feat, num_draw), requires_grad=True).float()
else:
    model_mal = torch.randn((num_feat, 1), requires_grad=True).float()
    model_fem = torch.randn((num_feat, 1), requires_grad=True).float()

In [21]:
print(model_mal.size())

torch.Size([16, 100000])


In [22]:
noise_m = torch.normal(0, 1, size=model_mal.size(), requires_grad=False).float()
noise_f = torch.normal(0, 1, size=model_fem.size(), requires_grad=False).float()
# model_mal = model_mal+noise_m
# model_fem = model_fem+noise_f
if mode == 'func' or mode =='fairdp':
    Q_mt = torch.from_numpy(Q_m)
    Q_ft = torch.from_numpy(Q_f)

In [23]:
def update_one_step(num_draw, model, coff, Q, noise, mode):
    if mode == 'fair' or mode == 'fairdp':
        coff_0, coff_1, coff_2 = coff
        model = model + noise
        loss = (1/num_draw)*(coff_0 + torch.sum(torch.mm(model.T, coff_1)) + torch.trace(torch.mm(torch.mm(model.T, coff_2), model)))
        model.retain_grad()
        loss.backward()
    else:
        coff_0, coff_1, coff_2 = coff
        model = model + noise
        loss = (1/num_draw)*(coff_0 + torch.mm(model.T, coff_1) + torch.mm(torch.mm(model.T, coff_2), model))
        model.retain_grad()
        loss.backward()
    return loss.item()

In [ ]:
tk0 = tqdm(range(epoch), total=epoch)
for ep in tk0:
    if mode == 'torch':
        loss_m = m_coff_0 + torch.mm(model_mal.T,m_coff_1) + torch.mm(torch.mm(model_mal.T.float(),m_coff_2.float()), model_mal) # + alpha*torch.norm(torch.mm(Q_mt, model_mal) - torch.mm(Q_ft, model_fem), p=2)
        loss_f = f_coff_0 + torch.mm(model_fem.T,f_coff_1) + torch.mm(torch.mm(model_fem.T.float(),f_coff_2.float()), model_fem) # + alpha*torch.norm(torch.mm(Q_mt, model_mal) - torch.mm(Q_ft, model_fem), p=2)
        model_mal.retain_grad()
        model_fem.retain_grad()
        loss_m.backward()
        loss_f.backward()
    elif mode == 'func':
        loss_m = m_coff_0 + torch.mm(torch.mm(Q_mt,model_mal).T,m_coff_1) + torch.mm(torch.mm(torch.mm(Q_mt,model_mal).T.float(),m_coff_2.float()), torch.mm(Q_mt,model_mal)) # + alpha*torch.norm(torch.mm(Q_mt, model_mal) - torch.mm(Q_ft, model_fem), p=2)
        loss_f = f_coff_0 + torch.mm(torch.mm(Q_ft,model_fem).T,f_coff_1) + torch.mm(torch.mm(torch.mm(Q_ft,model_fem).T.float(),f_coff_2.float()), torch.mm(Q_ft,model_fem)) # + alpha*torch.norm(torch.mm(Q_mt, model_mal) - torch.mm(Q_ft, model_fem), p=2)
        model_mal.retain_grad()
        model_fem.retain_grad()
        loss_m.backward()
        loss_f.backward()
    elif mode == 'fair':
        loss_mal = 0
        loss_fem = 0
        noise_m = torch.normal(0, 1, size=model_mal.size(), requires_grad=False).float()
        noise_f = torch.normal(0, 1, size=model_fem.size(), requires_grad=False).float()
        loss_m = update_one_step(num_draw=num_draw, model=model_mal, coff=(m_coff_0, m_coff_1, m_coff_2), Q=None, noise = noise_m, mode=mode)
        loss_f = update_one_step(num_draw=num_draw, model=model_fem, coff=(f_coff_0, f_coff_1, f_coff_2), Q=None, noise = noise_f, mode=mode)
        loss_mal = loss_m
        loss_fem = loss_f
        # loss_mal = loss_mal/num_draw
        # loss_fem = loss_fem/num_draw
    elif mode == 'fairdp':
        for i in range(num_draw):
            model_mal = model_mal + torch.normal(0, 1, size=model_mal.size(), requires_grad=False).float()
            model_fem = model_fem + torch.normal(0, 1, size=model_fem.size(), requires_grad=False).float()
            loss_m = m_coff_0 + torch.mm(torch.mm(Q_mt,model_mal).T,m_coff_1) + torch.mm(torch.mm(torch.mm(Q_mt,model_mal).T.float(),m_coff_2.float()), torch.mm(Q_mt,model_mal)) # + alpha*torch.norm(torch.mm(Q_mt, model_mal) - torch.mm(Q_ft, model_fem), p=2)
            loss_f = f_coff_0 + torch.mm(torch.mm(Q_ft,model_fem).T,f_coff_1) + torch.mm(torch.mm(torch.mm(Q_ft,model_fem).T.float(),f_coff_2.float()), torch.mm(Q_ft,model_fem)) # + alpha*torch.norm(torch.mm(Q_mt, model_mal) - torch.mm(Q_ft, model_fem), p=2)
            model_mal.retain_grad()
            model_fem.retain_grad()
            loss_m.backward()
            loss_f.backward()
    if ep%5 == 0:
        print(loss_mal, loss_fem)
    with torch.no_grad():
        model_mal -= lr * model_mal.grad
        model_fem -= lr * model_fem.grad
        # model = (model_mal + model_fem)/2
        # pred_m = torch.sigmoid(torch.mm(model_mal.T,torch.from_numpy(X_mal.astype(np.float32)).T))
        # pred_f = torch.sigmoid(torch.mm(model_fem.T,torch.from_numpy(X_fem.astype(np.float32)).T))
        # pred = torch.sigmoid(torch.mm(torch.from_numpy(X.astype(np.float32)), model))
#         pred = np.mean(pred.numpy(), axis=1)
#         pred_m = np.mean(pred_m.numpy(), axis=1)
#         pred_f = np.mean(pred_f.numpy(), axis=1)
        
#         print(pred.shape, pred_m.shape, pred_f.shape)
#         print("Train prediction:", pred)
#         print("Male prediction:", pred_m)
#         print("Female prediction:", pred_f)


        # acc_mal = accuracy_score(y_true=y_mal, y_pred=np.round(pred_m.reshape(-1,1)))
        # acc_fem = accuracy_score(y_true=y_fem, y_pred=np.round(pred_f.reshape(-1,1)))
        # acc = accuracy_score(y_true=y, y_pred=np.round(pred.reshape(-1,1)))
        # if ep%50 == 0:
        #     print("Epoch {}: loss male {}, loss female {}, acc male {}, acc female {}, train acc {}".format(ep, loss_m.item(), loss_f.item(), acc_mal, acc_fem, acc))            
    model_mal.grad = torch.zeros(model_mal.size())
    model_fem.grad = torch.zeros(model_fem.size())

  1%|▍                                       | 1/100 [03:19<5:28:31, 199.10s/it]

2.0135087966918945 2.0088343620300293


  3%|█▏                                      | 3/100 [09:53<5:19:48, 197.82s/it]

In [ ]:
# epsilon = [0.1, 1.0, 5.0, 10.0, 50.0]

In [ ]:
# # normal LR
# 0.9189058339385137
# # With taylor approximation
# 0.8900992495763738
# # With functional mechanism
# performance=[0.41636407649479545, 0.5833938513677076, 0.5952553861050593, 0.6056644880174292, 0.7780198499152747]
# # With functional mechanism and smooth classifier

In [ ]:
# plt.plot(range(len(epsilon)), performance, '-*', label = 'FM')
# plt.plot(range(len(epsilon)), np.ones(len(epsilon))*0.9189058339385137, '-o', label = 'no-DP')
# plt.plot(range(len(epsilon)), np.ones(len(epsilon))*0.8900992495763738, '-o', label = 'no-DP Taylor approx')
# plt.ylabel(r'Accuracy')
# plt.xlabel(r'$\epsilon$')
# plt.title('Bank dataset')
# plt.xticks(range(len(epsilon)), epsilon)
# plt.legend()

### 5.1 With scipy

In [ ]:
# X = female_df[feature_cols].values
# y = female_df[label].values.reshape(-1, 1)

# X = X/np.linalg.norm(X, ord=2, axis=1).reshape(-1, 1)
# coff_0, coff_1, coff_2 = get_coefficient(X=X, y=y, epsilon=10.0, mode='scipy_dp')
# print(coff_1.shape, coff_2.shape)

In [ ]:
# model = np.random.normal(0.0, 1.0, coff_1.shape)
# print(model.shape)

In [ ]:
# def sigmoid(x):
#     return np.exp(x)/(1 + np.exp(x))

# def f(w):
#     return coff_0 + np.squeeze(np.dot(coff_1.T, w)) + np.sum(coff_2*np.dot(w, w.T))

In [ ]:
# res = minimize(f, model, method='CG')
# print(res)

In [ ]:
# best_model = res['x'].reshape(-1,1)
# pred = sigmoid(np.dot(X, best_model))
# print(pred.shape)

In [ ]:
# acc = accuracy_score(y_true=y, y_pred=np.round(pred))
# print("Accuracy score: {}".format(acc))

In [ ]:
# X = train_df[feature_cols].values
# y = train_df[label].values.reshape(-1, 1)

# X = X/np.linalg.norm(X, ord=2, axis=1).reshape(-1, 1)
# pred = pred = sigmoid(np.dot(X, best_model))
# acc = accuracy_score(y_true=y, y_pred=np.round(pred))
# print("Accuracy score: {}".format(acc))